<a href="https://colab.research.google.com/github/rage-against-the-machine-learning/cs7643-DL-Group-Project/blob/faiss_densenet/faiss_densenet_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from google.colab.output import eval_js
def shrink(height="300"):
    eval_js(f'google.colab.output.setIframeHeight({height})')

Get the data - only need to run once

In [3]:
numpy_data_path = "/content/drive/MyDrive/numpy_images"

Set up environment

In [4]:
! pip install --upgrade pip
! pip install torch torchvision torchsummary gca pandas numpy matplotlib seaborn scikit-learn==0.23.2 pycocotools tqdm umap-learn kneed
! pip install git+https://github.com/waleedka/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI
# TODO: install the docem python package via github
! pip list
shrink()

  Cloning https://github.com/waleedka/cocoapi.git to /tmp/pip-install-tjlk09me/pycocotools_42c2e931bf6142d8b943b388fb95cb4f
  Running command git clone -q https://github.com/waleedka/cocoapi.git /tmp/pip-install-tjlk09me/pycocotools_42c2e931bf6142d8b943b388fb95cb4f
  fatal: could not read Username for 'https://github.com': No such device or address
ERROR: Could not find a version that satisfies the requirement pycocotools (unavailable) (from versions: 2.0.0, 2.0.1, 2.0.2a1, 2.0.2)
ERROR: No matching distribution found for pycocotools (unavailable)
Package                       Version
----------------------------- -------------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
appdirs                       1.4.4
argon2-cffi                   20.1.0
arviz                         0.11.2
astor                         0.8.1
astropy                       4.2.1
astunparse                   

Choose a GPU runtime

In [5]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(torch.cuda.current_device())
    print(torch.cuda.device_count())
    print(torch.cuda.get_device_name(0))
    
device

0
1
Tesla T4


device(type='cuda', index=0)

In [6]:
import os
import multiprocessing as mp
import random
import sys

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import torch 
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [8]:
import os
import json
print(os.getcwd())

os.chdir("/content/drive/MyDrive/numpy_images/")
print(os.listdir())

embeddings_path = "densenet_pretrained_validation.pt"
mapping_path = "densenet_pretrained_index2cocoIDs.json"
embeddings = torch.load(embeddings_path,map_location=torch.device('cuda'))

with open(mapping_path, 'r') as fp:
    idx2Img = json.load(fp)
    
idx2Img = dict(zip(list(map(int, idx2Img.keys())), list(idx2Img.values())))
#print(idx2Img)


/content
['densenet_pretrained_embs_len_50176.pt', 'data', 'annotations', 'resnext_embeddings.json', 'resnext_embeddings.pt', 'di_index2cocoIDs.json', 'di_embed_val_batch_41.pt', 'faiss.ipynb', 'id_imagecocoid_mapping.npy', 'val_img_emb.pkl', 'densenet_pretrained_index2cocoIDs.json', 'densenet_pretrained_validation.pt']


In [9]:
# Img Ids to query
random.seed(10)

query_images = [100582, 38210, 495291]
#TODO: Madhuri
#Commenting this out so that same images (as in Michael's) can be used for results
#query_images = random.sample(list(idx2Img.values()), 10)
#query_images = [558661, 123071, 535156, 185614, 8493, 538859, 10583, 254976, 176847, 369594]
print(query_images)
# Position of query images in embedding space
query_idxs = [key for key, value in idx2Img.items() if idx2Img[key] in query_images]

print("query image ids: ", query_images)
print("query image idxs: ", query_idxs)

[100582, 38210, 495291]
query image ids:  [100582, 38210, 495291]
query image idxs:  [1067, 6753, 18723]


In [10]:

# Update based on the location in your system
annot_file = "annotations/instances_val2014.json"


In [11]:
!pip install faiss-gpu

In [12]:
print(embeddings.shape)
em_np = embeddings.cpu().numpy()
print(type(em_np))

torch.Size([21002, 50176])
<class 'numpy.ndarray'>


In [13]:
query_embedding = em_np[query_idxs]
print(query_embedding.shape)

(3, 50176)


In [14]:
import faiss
index = faiss.IndexFlatL2(embeddings.shape[1])   # build the index
print(index.is_trained)
index.train(em_np)
index.add(em_np)                  # add vectors to the index
print(index.ntotal)

True
21002


In [22]:
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import numpy as np
import itertools
from pycocotools.coco import COCO
from mpl_toolkits.mplot3d import Axes3D
from collections import Counter
import pandas as pd

class EmbeddingPlotter:
    def __init__(self, annot_file, embeddings, idx2Img):
        self.annot_file = annot_file
        self.coco = COCO(self.annot_file)
        cats = self.coco.loadCats(self.coco.getCatIds())
        self.cat_id_to_name_dict = {cat['id']: cat['name'] for cat in cats}
        self.img_ids = list(idx2Img.values())
        self.annIds = self.coco.getAnnIds(self.img_ids)
        #self.labels = [self.coco.loadAnns(self.annIds)]
        
        
        
        
        self.embeddings = embeddings
        self.idx2Img = idx2Img
        self.labels = []
        for k, v in self.idx2Img.items():
            ann_ids_to_append = self.coco.getAnnIds(self.idx2Img[k])
            anns_to_append = self.coco.loadAnns(ann_ids_to_append)
            self.labels.append(anns_to_append)
            
        
        self.category_labels=[]
        # self.category_multi_labels = []
        # self.category_multi_labels_counts = []
        self.supercategory_labels = []
        # self.supercategory_multi_labels = []
        # self.supercategory_multi_labels_counts = []
        
        
 
        
        cats_dict = {cat['id']: cat for cat in cats}
        for label in self.labels:
            
            if len(label) > 0:
                categories = []
                supercategories = []
                
                for annotation in label:
                    categories.append(cats_dict[annotation['category_id']]['name'])
                    supercategories.append(cats_dict[annotation['category_id']]['supercategory'])
                # self.category_multi_labels_counts.append(categories)
                # self.category_multi_labels.append(list(set(categories)))
                # self.supercategory_multi_labels.append(list(set(supercategories)))
                # self.supercategory_multi_labels_counts.append(supercategories)
        
                # most common category among objects in the picture
                category, _ = Counter(categories).most_common(1)[0]
                supercategory, _ = Counter(supercategories).most_common(1)[0]
                
                self.category_labels.append(category)
                self.supercategory_labels.append(supercategory)
            else:
                # self.supercategory_multi_labels.append([])
                # self.supercategory_multi_labels_counts.append([])
                # self.category_multi_labels.append([])
                # self.category_multi_labels_counts.append([])
                self.category_labels.append(None)
                self.supercategory_labels.append(None)
        
    def plot_coco_images(self, img_ids, show_labels=True,
                         show_annotations=False, title_suffix=None, figtitle=None):
        coco = self.coco
        cat_id_to_name_dict = self.cat_id_to_name_dict
        nrows = len(img_ids)
        ncols = len(img_ids[0])
    
        fig = plt.figure(figsize=(ncols*6, nrows*6))
    
        for i, img_id in enumerate(itertools.chain(*img_ids)):
            ax = fig.add_subplot(nrows, ncols, i+1)
            img = coco.loadImgs(img_id)[0]
            I = io.imread(img['coco_url'])
    
            ax.imshow(I, aspect='equal', )
            ax.axis('off')
            
            if title_suffix is not None:
                title=f'{img_id}: {title_suffix[i//nrows, i%ncols]}'
            else:
                title = str(img_id)
            ax.set_title(title)
    
        
            annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
            anns = coco.loadAnns(annIds)
            if show_annotations:
                coco.showAnns(anns)
    
            if show_labels:
                for ann in anns:
                    [x, y, _, _] = ann['bbox']
                    label = cat_id_to_name_dict[ann['category_id']]
    
                    t=ax.annotate(label, (x,y), fontsize=12)
                    t.set_bbox(dict(facecolor='white', alpha=0.2, edgecolor='white'))
    
        plt.tight_layout()
        
        if figtitle:
            fig.suptitle(figtitle)
            plt.subplots_adjust(top=0.90)
        plt.show()
        
    def plot_neighbors(self, X_idx, nbrs, show_annotations=False):
        idx2Img = self.idx2Img
        embeddings = self.embeddings
        m, n = np.shape(embeddings)
        img_id = idx2Img[X_idx]
        
        distances, indices = nbrs.kneighbors(embeddings[X_idx, :].reshape(1, n), 16)
        distances, indices = distances[0], indices[0]
        neighbors_ids = np.array(list(map(idx2Img.get, indices)))
        np.insert(neighbors_ids, 0, img_id)
        np.insert(distances, 0, 0)
        
        title_suffix = np.array([f'distance={distance:.2f}' for distance in distances])
        figtitle=f'Neighbors of {img_id}'
        self.plot_coco_images(neighbors_ids.reshape(4, 4).tolist(), 
                         title_suffix=title_suffix.reshape(4,4),
                         show_annotations=show_annotations,
                         figtitle=figtitle)
        
    def plot_3d(self, X, title=None, columns=['x0', 'x1', 'x2'], **kwargs):
        labels = self.supercategory_labels
        
        
        
        df = pd.DataFrame(X, columns=columns)
        df['label'] = labels
        cmap = plt.get_cmap('gist_rainbow')
        unique_labels = list(set(labels))
        labels_dict = {label: i for i, label in enumerate(unique_labels)}
        num_colors = len(unique_labels)
        c = [cmap(1.*labels_dict[label]/num_colors) for label in labels]
    
        df['c'] = c
        df = df[~df['label'].isna()]
        ax = plt.figure(figsize=(16,10)).gca(projection='3d')
        
        for label, grp in df.groupby('label'):
            ax.scatter(
            xs=grp[columns[0]], 
            ys=grp[columns[1]], 
            zs=grp[columns[2]], 
            c=grp['c'], 
            label=label,
            **kwargs
        #     cmap='gist_rainbow'
        )
        
        ax.set_xlabel(columns[0])
        ax.set_ylabel(columns[1])
        ax.set_zlabel(columns[2])
        
        plt.legend(loc='best')
        if title:
            plt.title(title)
        plt.show()

In [ ]:
plotter = EmbeddingPlotter(annot_file = annot_file, embeddings=embeddings, idx2Img=idx2Img)

loading annotations into memory...


In [ ]:
k = 11                         # we want to see 10 nearest neighbors
D, I = index.search(query_embedding, k) # sanity check
print(I)
print(D)

In [ ]:
for j in range(I.shape[0]):
  l = []
  for i in range(I.shape[1]):
    l.append(idx2Img[I[j][i]])
  plotter.plot_coco_images([l])


In [1]:
from operator import itemgetter
def faiss_precision (plotter, faiss_obj, query_embedding, query_idx, idx2coco_map: dict,
                     n_neighbors: int = 11, at_k: int = 4, category=True):
    

    D, indices = faiss_obj.search(query_embedding, n_neighbors)
    #closest_at_k = closest[1: at_k + 1]
    #indices = I[0][1:k+1]
    retrieved_indices = indices[0][1:at_k+1] # top 4 image locations

    if category == True:
        ground_truth = plotter.category_labels[query_idx]
        retrieved_labels = itemgetter(*retrieved_indices)(plotter.category_labels)
    else:
        ground_truth = plotter.supercategory_labels[query_idx]
        retrieved_labels = itemgetter(*retrieved_indices)(plotter.supercategory_labels)
    
    precision = np.array([1 if i == ground_truth else 0 for i in retrieved_labels]).sum() / (n_neighbors-1)
    return precision

def avg_precision(plotter, faiss, em_np, query_idxs, idx2Img, n_neighbors, k, category=True):
    avg_precision = 0
    for idx in query_idxs:
      embed = em_np[idx]
      embed = np.reshape(embed, (1,25088))
      avg_precision += faiss_precision(plotter, faiss, embed, idx, idx2Img, n_neighbors, k, category)
    avg_precision = avg_precision / len(query_idxs)
    return avg_precision


In [2]:
category_precision = avg_precision(plotter, index, em_np, query_idxs, idx2Img, 11, k=4, category=False)
supercategory_precision = avg_precision(plotter, index, em_np, query_idxs, idx2Img, 11, k=4, category=True)

print("Category-level precision: {}%".format(round(category_precision*100, 2)))
print("Supercategory-level precision: {}%".format(round(supercategory_precision*100, 2)))

NameError: ignored

In [ ]:
def faiss_recall(plotter, faiss_obj, query_embedding, query_idx, idx2coco_map: dict,
                     n_neighbors: int = 11, at_k: int = 4, category=True):
    """
    :plotter: plotter object from plot_utils.py
    :idx: embedding nidex
    :nbrs: for nearest neighbors object
    :category: if True then calculate on categories, otherwise calcuate at supercategory
    """

    D, indices = faiss_obj.search(query_embedding, n_neighbors)
    #closest_at_k = closest[1: at_k + 1]
    retrieved_indices = indices[0][1:at_k+1] # top 4 image locations
    
    if category == True:
        ground_truth = plotter.category_labels[query_idx]
        total_retrieved_labels = itemgetter(*indices[0][1:])(plotter.category_labels)
        retrieved_labels = itemgetter(*retrieved_indices)(plotter.category_labels)
    else:
        ground_truth = plotter.supercategory_labels[query_idx]
        total_retrieved_labels = itemgetter(*indices[0][1:])(plotter.supercategory_labels)
        retrieved_labels = itemgetter(*retrieved_indices)(plotter.supercategory_labels)
    relevant_labels = [i for i in total_retrieved_labels if i == ground_truth]
    recall = np.array([1 if i == ground_truth else 0 for i in retrieved_labels]).sum() / len(relevant_labels)
    if len(relevant_labels) == 0:
        return 0
    else:
        return recall

def avg_recall(plotter, faiss, em_np, query_idxs, idx2Img, n_neighbors, k, category=True):
    avg_recall = 0
    for idx in query_idxs:
        embed = em_np[idx]
        embed = np.reshape(embed, (1,25088))
        avg_recall += faiss_recall(plotter, faiss, embed, idx, idx2Img, n_neighbors, k, category)

    avg_recall = avg_recall / len(query_idxs)
    return avg_recall

In [ ]:
category_recall = avg_recall(plotter, index, em_np, query_idxs, idx2Img, 11, k=4, category=False)
supercategory_recall = avg_recall(plotter, index, em_np, query_idxs, idx2Img, 11, k=4, category=True)

print("Category-level recall: {}%".format(round(category_recall*100, 2)))
print("Supercategory-level recall: {}%".format(round(supercategory_recall*100, 2)))

Category-level recall: 100.0%
Supercategory-level recall: 66.67%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in long_scalars
